In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/비버웍스_샘플데이터_open.xlsx', sheet_name=None)
dirty = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/해커톤참고자료(dirty).xlsx')

In [ ]:
orderQueen = df['오더퀸']
unos = df['우노스']

# 전처리
### 모든 데이터에 적용 가능해야 함
- 동일한 데이터프레임 Form으로 변경
- 취사선택 기능 (제거할 특수문자 입력)
- 동일한 기준 설정 (영어, 단위 제거)
- 오탈자 교정 (자모 단위)
- 띄어쓰기 교정 (soynlp - soyspacing)


## OrderQueen

In [ ]:
orderQueen.info()

In [ ]:
orderQueen.duplicated().sum()

In [ ]:
orderQueen.head()

In [ ]:
orderQueen.tail()

# Unos

In [ ]:
unos.info()

In [ ]:
unos[unos['카테고리명'].isnull()]

In [ ]:
unos['메뉴코드'].duplicated().sum()

In [ ]:
unos.head()

In [ ]:
unos.tail()

# Dirty Sample

In [ ]:
dirty.info()

In [ ]:
dirty